In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import xarray as xr
import geopandas as gpd

In [2]:
#os.chdir('../data/anuario_aforos')

In [5]:
cuenca = 'ebro'

In [14]:
start = '1980-10-01'
end = '2020-09-30'

In [15]:
area_min = 100
area_max = None

In [16]:
disponibilidad = .9 # % del periodo de estudio

### Estaciones

In [38]:
estaciones = gpd.read_file('../data/GIS/estaciones_seleccion_EFAS.shp')
estaciones.indroea = estaciones.indroea.astype(int)
estaciones.suprest = estaciones.suprest.astype(int)
estaciones.set_index('indroea', drop=True, inplace=True)
estaciones = estaciones.loc[estaciones.cuenca == cuenca]

estaciones.shape

(61, 27)

### Caudal

In [29]:
# cargar series temporales
data = pd.read_csv(f'../data/anuario_aforos/{cuenca}/afliq.csv', sep=';', index_col='indroea')
data.fecha = pd.to_datetime(data.fecha, dayfirst=True)

# recortar a la fecha de estudio
data = data.loc[(data.fecha >= start) & (data.fecha <= end),:]

# reformatear series de caudal y nivel
Q = pd.DataFrame(index=pd.date_range(data.fecha.min(), data.fecha.max(), freq='1d'), columns=data.index.unique(), dtype=float) 
for stn in Q.columns:
    data_stn = data.loc[stn].set_index('fecha', drop=True)
    Q[stn] = data_stn.caudal

# convertir nombre de las columnas en texto
Q.columns = [str(col) for col in Q.columns]

# eliminar estaciones sin ningún dato
Q.dropna(axis=1, how='all', inplace=True)

print('{0} estaciones con series de caudal en el {1}'.format(Q.shape[1], cuenca.capitalize()))

# convertir los nombres de las columnas en enteros
Q.columns = [int(stn) for stn in Q.columns]

262 estaciones con series de caudal en el Ebro


In [31]:
Q = Q[estaciones.index]

In [32]:
estaciones.suprest

indroea
9002    25194
9003     1445
9004     2759
9005     5469
9011    40434
        ...  
9189      217
9190      160
9191      534
9197      283
9221       22
Name: suprest, Length: 61, dtype: object

In [41]:
3600 * 24

86400

In [46]:
Qespecifico = Q / estaciones.suprest * 3600 * 24 * 10e-3 # mm/d
Qespecifico.index.name = 'date'

In [47]:
Qespecifico.to_csv(f'../data/anuario_aforos/{cuenca}/caudal_especifico.csv', float_format='%.3f')